In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
from scipy import sparse

from google.colab import files
uploaded =files.upload()

1. Verifique os tipos de atributos presentes em seu dataset e classifique os entre numérico, categórico ou textual.

In [ ]:
# realiza a leitura e o filtro do dataset para utilizar apenas as colunas desejadas
df = pd.read_csv('darknet.csv', usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 20, 21, 24, 25, 26, 29, 30, 31, 34, 35, 36, 40, 41, 42, 43, 44, 45, 49, 50, 51, 52, 53, 57, 58, 59, 60, 65, 66, 67, 68, 70, 71, 72, 73, 74, 81, 82, 83, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, 20, 21, 24, 25, 26, 29, 30, 31, 34, 35, 36, 40, 41, 42, 43, 44, 45, 49, 50, 51, 52, 53, 57, 58, 59, 60, 65, 66, 67, 68, 70, 71, 72, 73, 74, 81, 82, 83, 84])
# descarta linha com pelo menos um inf ou nan
any_inf_or_nan = df.isin([np.inf, -np.inf, np.nan]).any(axis='columns')
df = df[~any_inf_or_nan]
df_textual = df.select_dtypes(include=['object'])
TEXTUAL_ATTIBUTES = df_textual.columns.tolist()
df_number = df.select_dtypes(exclude=['object'])
NUMBER_ATRIBUTES = df_number.columns.tolist()

NUMBER_ATRIBUTES


FileNotFoundError: ignored

2. Para cada tipo de atributo, crie um script que extraia as características
desses atributos.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# extração de características de atributos textuais
def extraction_characteristic_textual(df_train_data, df_test_data):
  vectorizer = TfidfVectorizer(max_features=100)
  # train vectorizer
  vectorizer.fit(df_train_data)
  # transform train and test data to features
  train_features = vectorizer.transform(df_train_data)
  test_features = vectorizer.transform(df_test_data)
  return train_features, test_features



3. Crie um script que junte esses atributos em uma única matriz.

In [ ]:
def matrix_features_attributes(df_textual, df_number):
  mid = int((len(df_textual) + 1)/2)
  # split data into train and test
  df_train_data = df_textual[mid:]
  df_test_data = df_textual[:mid]

  df_train_data_number = df_number[mid:]
  df_test_data_number = df_number[:mid]

  train_number = df_train_data_number.values
  test_number = df_test_data_number.values
  # para cada atributo, é realizado o cálculo
  train_features_textual, test_features_textual = extraction_characteristic_textual(df_train_data[TEXTUAL_ATTIBUTES[0]], df_test_data[TEXTUAL_ATTIBUTES[0]])
  train_textual = train_features_textual.toarray()
  test_textual = test_features_textual.toarray()

  for t in TEXTUAL_ATTIBUTES[1:]:
    train_features_textual, test_features_textual = extraction_characteristic_textual(df_train_data[t], df_test_data[t])
    train_textual = sparse.hstack((train_textual, train_features_textual))
    test_textual = sparse.hstack((test_textual, test_features_textual))


  train_features_total = sparse.hstack((train_number, train_textual))
  test_features_total = sparse.hstack((test_number, test_textual))
  return train_features_total, test_features_total

train_features_total,test_features_total = matrix_features_attributes(df_textual, df_number)
train_features_total.shape, test_features_total.shape

4. Crie um script que normalize a matriz resultante do passo anterior.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

def normalization_min_max_scaling(train_features, test_features):
  # initialize min max
  scaler = MinMaxScaler()
  # train min max
  scaler.fit(train_features)
  # transform features
  train_features_norm = scaler.transform(train_features)
  test_features_norm = scaler.transform(test_features)
  return train_features_norm, test_features_norm

def normalization_standardization(train_features, test_features):
  # initialize min max
  scaler = StandardScaler()
  # train min max
  scaler.fit(train_features)
  # transform features
  train_features_norm = scaler.transform(train_features)
  test_features_norm = scaler.transform(test_features)
  return train_features_norm, test_features_norm

train_features_norm, test_features_norm = normalization_min_max_scaling(train_features_total.toarray(), test_features_total.toarray())
#train_features_norm, test_features_norm = normalization_standardization(train_features_total.toarray(), test_features_total.toarray())
train_features_norm.shape, test_features_norm.shape